# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [51]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from datetime import datetime
from typing import Optional, Union
import uuid
import csv

In [52]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
# MODEL = "llama3.2"
# openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')


OpenAI API Key exists and begins sk-proj-


In [53]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [54]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content

gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7879
* To create a public link, set `share=True` in `launch()`.


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [55]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499", "madrid":"$299"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [56]:
get_ticket_price("Berlin")

Tool get_ticket_price called for Berlin


'$499'

In [57]:
# make a second tool to book flights
def book_flight(
    departure_city: str,
    destination_city: str,
    departure_date: Union[str, datetime],
    is_round_trip: bool = False,
    return_date: Optional[Union[str, datetime]] = None,
    csv_filename: str = "flight_bookings.csv"
) -> dict:
    """
    Simulate airline flight booking and write booking information to a CSV file.
    
    Args:
        departure_city (str): Name of the departure city
        destination_city (str): Name of the destination city
        departure_date (str or datetime): Departure date (YYYY-MM-DD format if string)
        is_round_trip (bool): True for round-trip, False for one-way
        return_date (str or datetime, optional): Return date (required for round-trip)
        csv_filename (str): Name of the CSV file to write to
    
    Returns:
        dict: Booking confirmation with booking ID and details
        
    Raises:
        ValueError: For invalid input parameters
        TypeError: For incorrect data types
    """
    
    # Input validation
    if not departure_city or not isinstance(departure_city, str):
        raise ValueError("Departure city must be a non-empty string")
    
    if not destination_city or not isinstance(destination_city, str):
        raise ValueError("Destination city must be a non-empty string")
    
    if departure_city.strip().lower() == destination_city.strip().lower():
        raise ValueError("Departure and destination cities cannot be the same")
    
    # Date parsing and validation
    def parse_date(date_input, field_name):
        if isinstance(date_input, datetime):
            return date_input
        elif isinstance(date_input, str):
            try:
                return datetime.strptime(date_input.strip(), "%Y-%m-%d")
            except ValueError:
                raise ValueError(f"{field_name} must be in YYYY-MM-DD format")
        else:
            raise TypeError(f"{field_name} must be a string or datetime object")
    
    # Parse departure date
    parsed_departure_date = parse_date(departure_date, "Departure date")
    
    # Validate departure date is not in the past
    today = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
    if parsed_departure_date < today:
        raise ValueError("Departure date cannot be in the past")
    
    # Round-trip validation
    parsed_return_date = None
    if is_round_trip:
        if return_date is None:
            raise ValueError("Return date must be specified for round-trip flights")
        
        parsed_return_date = parse_date(return_date, "Return date")
        
        if parsed_return_date <= parsed_departure_date:
            raise ValueError("Return date must be after departure date")
    elif return_date is not None:
        raise ValueError("Return date should not be provided for one-way flights")
    
    # Generate booking ID
    booking_id = f"FL{uuid.uuid4().hex[:8].upper()}"
    
    # Create booking record
    booking_record = {
        'booking_id': booking_id,
        'departure_city': departure_city.strip().title(),
        'destination_city': destination_city.strip().title(),
        'departure_date': parsed_departure_date.strftime("%Y-%m-%d"),
        'is_round_trip': is_round_trip,
        'return_date': parsed_return_date.strftime("%Y-%m-%d") if parsed_return_date else '',
        'booking_timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    
    # Write to CSV file
    try:
        # Check if file exists to determine if we need headers
        file_exists = os.path.isfile(csv_filename)
        
        with open(csv_filename, 'a', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['booking_id', 'departure_city', 'destination_city', 
                         'departure_date', 'is_round_trip', 'return_date', 'booking_timestamp']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            
            # Write header if file is new
            if not file_exists:
                writer.writeheader()
            
            # Write the booking record
            writer.writerow(booking_record)
            
    except IOError as e:
        raise IOError(f"Error writing to CSV file: {e}")
    
    # Return confirmation
    confirmation = {
        'status': 'success',
        'message': 'Flight booked successfully',
        'booking_id': booking_id,
        'departure_city': booking_record['departure_city'],
        'destination_city': booking_record['destination_city'],
        'departure_date': booking_record['departure_date'],
        'flight_type': 'Round-trip' if is_round_trip else 'One-way',
        'return_date': booking_record['return_date'] if is_round_trip else None,
        'csv_file': csv_filename
    }
    
    return confirmation
        

In [58]:
# There's a particular dictionary structure that's required to describe our function:

price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}
booking_function = {
    "name": "book_flight",
    "desription": "Book a flight and write booking information to an external database. Call this whenever you need to book a flight after a customer instructs you to do so.",
    "parameters": {
        "type":"object",
        "properties": {
            "departure_city": { "type": "string", "description": "Name of the departure city"},
            "destination_city": { "type": "string", "description": "Name of the destination city"},
            "departure_date": { "type": "string", "description": "Departure date in YYYY-MM-DD format"},
            "is_round_trip": { "type": "string", "description": "True for round-trip, False for one-way"},
            "return_date": { "type": "string", "description" : "Return date in YYYY-MM-DD format, required for round-trips"},
        },
        "required": ["departure_city", "destination_city", "departure_date","is_round_trip"],
        "additionalProperties": False
    }
}


In [59]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function},
        {"type": "function", "function": booking_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [60]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [61]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    if tool_call.function.name=='get_ticket_price' :
        arguments = json.loads(tool_call.function.arguments)
        city = arguments.get('destination_city')
        price = get_ticket_price(city)
        response = {
            "role": "tool",
            "content": json.dumps({"destination_city": city,"price": price}),
            "tool_call_id": tool_call.id
        }
        return response, city
    elif tool_call.function.name=='book_flight':
        arguments= json.loads(tool_call.function.arguments)
        departure_city = arguments.get('departure_city')
        destination_city = arguments.get('destination_city')
        departure_date = arguments.get('departure_date')
        is_round_trip = arguments.get('is_round_trip')=='True'
        if is_round_trip:
            return_date = arguments.get('return_date')
        else: 
            return_date = None
        confirmation = book_flight( departure_city = departure_city, destination_city = destination_city,
                                    departure_date = departure_date, is_round_trip = is_round_trip, return_date=return_date)
        response = {
            "role": "tool",
            "content": json.dumps(confirmation),
            "tool_call_id":tool_call.id
        }
        return response, confirmation
    else:
        return "Unknown tool"

In [62]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7880
* To create a public link, set `share=True` in `launch()`.
